In [7]:
### I WILL PUT THE SOLVER CODE HERE AND WE CAN MERGE TO THE MAIN FILE ONCE ITS WORKING
#IMPORT DEPENDENCIES
import pandas as pd
import yfinance as yf
import datetime
import numpy as np

In [2]:
#APIs
# Define the list of stock ticker symbols
tickers = ['AGG', 'BNDX', 'SPY', 'IVW', 'IVE', 'BTC-USD', 'ETH-USD']

# Calculate the start date (three years ago from today)
start_date = datetime.datetime.now() - datetime.timedelta(days=1095)

# Initialize an empty dataframe to store the Close columns for each ticker
new_df = pd.DataFrame()

# Initialize an empty list to store the column names of Close prices for each ticker
ticker_closes = []

In [3]:
#Past 5 years of data into data frames
# Iterate over each ticker symbol
for ticker in tickers:
    # Fetch historical stock price data for the current ticker
    stock_data = yf.download(ticker, start=start_date, end=datetime.datetime.now())
    
    # Extract the 'Close' column and rename it for the current ticker
    close_column = stock_data[['Close']].rename(columns={'Close': f'{ticker}_Close'})
    
    # Compute the daily returns and insert them to the left of the corresponding ticker's Close column
    daily_returns = close_column.pct_change().rename(columns={f'{ticker}_Close': f'{ticker}_Daily_Return'})
    new_df = pd.concat([daily_returns, new_df], axis=1)
    
    # Append the Close column name for the current ticker to the ticker_closes list
    ticker_closes.append(f'{ticker}_Close')

# Drop rows containing NaNs from the new_df dataframe
new_df.dropna(inplace=True)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
## Feel free to adjust or send feedback if the code could be better or be more precise to what is needed:

# Calculate the annualized mean return for each asset
annualized_mean_return = new_df.mean() * 252

# Calculate the annualized standard deviation (risk) for each asset
annualized_std_dev = new_df.std() * (252 ** 0.5)

# Display annualized risk and return for each asset
for ticker in tickers:
    print(f'{ticker}:')
    print(f'Annualized Mean Return: {annualized_mean_return[f"{ticker}_Daily_Return"]:.2%}')
    print(f'Annualized Risk (Standard Deviation): {annualized_std_dev[f"{ticker}_Daily_Return"]:.2%}')
    print()

print(new_df)

AGG:
Annualized Mean Return: -5.88%
Annualized Risk (Standard Deviation): 6.82%

BNDX:
Annualized Mean Return: -5.22%
Annualized Risk (Standard Deviation): 6.03%

SPY:
Annualized Mean Return: 7.51%
Annualized Risk (Standard Deviation): 17.44%

IVW:
Annualized Mean Return: 7.38%
Annualized Risk (Standard Deviation): 21.37%

IVE:
Annualized Mean Return: 7.51%
Annualized Risk (Standard Deviation): 15.20%

BTC-USD:
Annualized Mean Return: -1.96%
Annualized Risk (Standard Deviation): 56.43%

ETH-USD:
Annualized Mean Return: -4.52%
Annualized Risk (Standard Deviation): 70.91%

            ETH-USD_Daily_Return  BTC-USD_Daily_Return  IVE_Daily_Return  \
Date                                                                       
2021-05-04             -0.051720             -0.067600          0.001695   
2021-05-05              0.082724              0.076696          0.002640   
2021-05-06             -0.009056             -0.017893          0.008911   
2021-05-07             -0.001762          

In [37]:
# Create a dictionary with calculated values
assets_data = {
    'Ticker': tickers,
    'Annualized_Mean_Return': [annualized_mean_return[f"{ticker}_Daily_Return"] for ticker in tickers],
    'Annualized_Std_Dev': [annualized_std_dev[f"{ticker}_Daily_Return"] for ticker in tickers]
}

# Convert the dictionary into a DataFrame
individual_assets = pd.DataFrame(assets_data)

# Print the DataFrame
print(individual_assets)

    Ticker  Annualized_Mean_Return  Annualized_Std_Dev
0      AGG               -0.058799            0.068186
1     BNDX               -0.052236            0.060349
2      SPY                0.075149            0.174400
3      IVW                0.073793            0.213682
4      IVE                0.075071            0.152028
5  BTC-USD               -0.019588            0.564345
6  ETH-USD               -0.045154            0.709100


In [20]:
agg_mean_return = individual_assets['Annualized_Mean_Return']

from scipy.optimize import minimize

# Define the objective function to minimize
def objective_function(weights, mean_returns, target_return):
    # Calculate the portfolio return
    portfolio_return = np.dot(mean_returns, weights)
    # Calculate the squared difference between the portfolio return and the target return
    return (portfolio_return - target_return) ** 2

# Define initial guess for weights
initial_guess = np.ones(len(agg_mean_return)) / len(agg_mean_return)  # Equal weights initially

# Define the target return
target_return = 0.05  # Example target return (replace with your desired target return)

# Perform optimization
result = minimize(objective_function, initial_guess, args=(agg_mean_return, target_return), method='SLSQP', bounds=[(0, 1)] * len(agg_mean_return), constraints={'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

# Print optimization result
print("Optimization Result:")
print(result)
print("Optimal solution:", result.x)



Optimization Result:
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1.6617205083896016e-20
       x: [ 2.960e-02  4.092e-02  2.605e-01  2.582e-01  2.604e-01
            9.721e-02  5.313e-02]
     nit: 4
     jac: [ 6.668e-11  5.413e-11  6.478e-11  6.212e-11  6.462e-11
            1.077e-11  4.202e-11]
    nfev: 32
    njev: 4
Optimal solution: [0.02960394 0.0409195  0.26053724 0.2581996  0.26040383 0.09720661
 0.05312929]


In [31]:
five_perc_return = pd.DataFrame(result.x)
five_perc_return

,0
0,0.029604
1,0.040919
2,0.260537
3,0.258200
4,0.260404
5,0.097207
6,0.053129


In [38]:
import pandas as pd

# Assuming individual_assets is your DataFrame
individual_assets['five_perc_return_weights'] = five_perc_return

# Now individual_assets will have a new column 'test_weights'
print(individual_assets)


    Ticker  Annualized_Mean_Return  Annualized_Std_Dev  \
0      AGG               -0.058799            0.068186   
1     BNDX               -0.052236            0.060349   
2      SPY                0.075149            0.174400   
3      IVW                0.073793            0.213682   
4      IVE                0.075071            0.152028   
5  BTC-USD               -0.019588            0.564345   
6  ETH-USD               -0.045154            0.709100   

   five_perc_return_weights  
0                  0.029604  
1                  0.040919  
2                  0.260537  
3                  0.258200  
4                  0.260404  
5                  0.097207  
6                  0.053129  


In [39]:
import numpy as np

# Calculate the portfolio return using sum product
portfolio_return_fice_perc = np.dot(individual_assets['five_perc_return_weights'], individual_assets['Annualized_Mean_Return'])

print("Portfolio Return:", portfolio_return_fice_perc)


Portfolio Return: 0.049999999871092264


In [40]:
agg_mean_return = individual_assets['Annualized_Mean_Return']

from scipy.optimize import minimize

# Define the objective function to minimize
def objective_function(weights, mean_returns, target_return):
    # Calculate the portfolio return
    portfolio_return = np.dot(mean_returns, weights)
    # Calculate the squared difference between the portfolio return and the target return
    return (portfolio_return - target_return) ** 2

# Define initial guess for weights
initial_guess = np.ones(len(agg_mean_return)) / len(agg_mean_return)  # Equal weights initially

# Define the target return
target_return = 0.1  # Example target return (replace with your desired target return)

# Perform optimization
result = minimize(objective_function, initial_guess, args=(agg_mean_return, target_return), method='SLSQP', bounds=[(0, 1)] * len(agg_mean_return), constraints={'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

# Print optimization result
print("Optimization Result:")
print(result)
print("Optimal solution:", result.x)



Optimization Result:
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.0006413505963976324
       x: [ 0.000e+00  0.000e+00  3.350e-01  3.302e-01  3.348e-01
            2.776e-17  8.114e-18]
     nit: 8
     jac: [ 2.978e-03  2.646e-03 -3.806e-03 -3.738e-03 -3.802e-03
            9.921e-04  2.287e-03]
    nfev: 64
    njev: 8
Optimal solution: [0.00000000e+00 0.00000000e+00 3.35046106e-01 3.30185200e-01
 3.34768694e-01 2.77555756e-17 8.11434823e-18]


In [41]:
ten_perc_return = pd.DataFrame(result.x)
ten_perc_return

,0
0,0.000000e+00
1,0.000000e+00
2,3.350461e-01
3,3.301852e-01
4,3.347687e-01
5,2.775558e-17
6,8.114348e-18
